In [ ]:
import requests
import cv2
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
import onnx
import tensorflow as tf
import keras
import tf2onnx
import onnxruntime

# Load Arial font and save as global variable
FONT = ImageFont.truetype("./BEATTECH.otf", 16)

# Load Keras model
keras_model = keras.models.load_model('./models/fer2013_mini_XCEPTION.99-0.65.hdf5', compile=False)

# Convert Keras model to ONNX model
onnx_model, _ = tf2onnx.convert.from_keras(keras_model)

# Save ONNX model to file
onnx_model_path = '/content/ONNX_fer2013_mini_XCEPTION.99-0.65.onnx'
onnx.save_model(onnx_model, onnx_model_path)

# Create ONNX Runtime session
ort_session = onnxruntime.InferenceSession(onnx_model_path)

# Define sentiment labels and colors
SENTIMENT_LABELS = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral', 'Uncertain']
SENTIMENT_COLORS = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0), (255, 0, 255), (0, 255, 255), (128, 128, 128), (192, 192, 192)]

# Define function analyze sentiment image
def analyze_image(url):
    # Download image from URL and detect faces
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img_draw = ImageDraw.Draw(img)
    img_array = np.array(img.convert('RGB'))
    gray = cv2.cvtColor(img_array, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)

    # Analyze sentiment for each detected face
    for (x, y, w, h) in faces:
        # Crop and transform face image
        face_img = Image.fromarray(gray[y:y+h, x:x+w])
        face_img = transforms.Resize(64)(face_img)
        face_img = transforms.CenterCrop(64)(face_img)
        face_img = transforms.ToTensor()(face_img)
        face_img = transforms.Normalize(mean=[0.5], std=[0.5])(face_img)
        face_img = face_img.unsqueeze(0).permute(0, 2, 3, 1)  # Change the input shape to (batch_size, 64, 64, 1)

        # Make prediction on face using the ONNX Runtime session
        input_name = ort_session.get_inputs()[0].name
        output_name = ort_session.get_outputs()[0].name
        output = ort_session.run([output_name], {input_name: face_img.numpy()})[0]
        scores = output[0]
        label = np.argmax(scores)

        # Check if prediction is uncertain based on threshold
        if np.max(scores) - np.sort(scores)[-2] < 0.2:
            label = 7 # Uncertain label

        # Draw rectangle around face and label with sentiment
        img_draw.rectangle([(x, y), (x+w, y+h)], outline=SENTIMENT_COLORS[label])
        img_draw.text((x, y-16), SENTIMENT_LABELS[label], fill=SENTIMENT_COLORS[label], font=FONT)

    # Display image with detected faces and labels
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Test the function on an example image
# url = 'https://faceswapper.ai/assets/img/sample/female-face-1.jpg'
url = 'https://cdn.psychologytoday.com/sites/default/files/styles/article-inline-half/public/blogs/35494/2014/04/148432-151140.jpg?itok=wEjXMHNI'
analyze_image(url)